In [ ]:
!apt-get install -y nvidia-cuda-toolkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libaccinj64-11.5 libbabeltrace1 libcub-dev libcublas11 libcublaslt11 libcudart11.0 libcufft10
  libcufftw10 libcuinj64-11.5 libcupti-dev libcupti-doc libcupti11.5 libcurand10 libcusolver11
  libcusolvermg11 libcusparse11 libdebuginfod-common libdebuginfod1 libegl-dev libgail-common
  libgail18 libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglvnd-core-dev libglvnd-dev
  libglx-dev libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libipt2 libnppc11 libnppial11 libnppicc11
  libnppidei11 libnppif11 libnppig11 libnppim11 libnppist11 libnppisu11 libnppitc11 libnpps11
  libnvblas11 libnvidia-compute-495 libnvidia-compute-510 libnvidia-compute-535 libnvidia-ml-dev
  libnvjpeg11 libnvrtc-built

In [ ]:
 !apt install -y pkg-config libopencv-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4+jammy0).
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'apt autoremove' to remove them.
The following packages will be REMOVED:
  pkgconf r-base-dev
The following NEW packages will be installed:
  pkg-config
0 upgraded, 1 newly installed, 2 to remove and 49 not upgraded.
Need to get 48.2 kB of archives.
After this operation, 11.3 kB disk space will be freed.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pkg-config amd64 0.29.2-1ubuntu3 [48.2 kB]
Fetched 48.2 kB in 1s (44.1 kB/s)
(Reading database ... 123629 files and directories currently installed.)
Removing r-base-dev (4.4.2-1.2204.0) ...
dpkg: pkgconf: dependency problems, but removing anyway as you requested:
 libsndfile1-dev:amd64 depends on pkg-config; however:
  Package pkg-config i

In [ ]:
!apt install -y ffmpeg libsm6 libxext6 libpng-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libpng-dev is already the newest version (1.6.37-3build5).
libsm6 is already the newest version (2:1.2.3-1build2).
libxext6 is already the newest version (2:1.3.4-1build1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
%%writefile example.cu
#include <sys/stat.h>
#include <sys/types.h>
#include <cstdio>
#include <cstring>
#include <iostream>
#include <cuda_runtime.h>
#include <opencv2/opencv.hpp>
#include "opencv2/imgproc/imgproc.hpp"
#include "opencv2/highgui/highgui.hpp"
#include <opencv2/core/core.hpp>
#include <chrono>
#include <unistd.h>

#define CUDA_DEBUG
#ifdef CUDA_DEBUG
#define CUDA_CHECK_ERROR(err) \
if (err != cudaSuccess) { \
    printf("Cuda error: %s\n", cudaGetErrorString(err)); \
    printf("Error in file: %s, line: %s\n", "FILE", "__LINE__"); \
}
#else
#define CUDA_CHECK_ERROR(err)
#endif

#define GAUSS(x, sigma) \
1 / sqrt(2 * M_PI * sigma * sigma) * exp(-x * x / (2 * sigma * sigma))


#define BLOCK_SIZE 16
#define FILTER_SIZE 3
#define TILE_SIZE (BLOCK_SIZE - FILTER_SIZE + 1)
#define M_PI 3.14159265358979323846

__global__ void blurFilterShared(const unsigned char* input, unsigned char* output, int width, int height, int channels) {

    extern __shared__ unsigned char sharedMem[];
    int sharedWidth = blockDim.x + FILTER_SIZE - 1;
    int sharedHeight = blockDim.y + FILTER_SIZE - 1;

    int x = blockIdx.x * TILE_SIZE + threadIdx.x;
    int y = blockIdx.y * TILE_SIZE + threadIdx.y;

    int sharedX = threadIdx.x + FILTER_SIZE / 2;
    int sharedY = threadIdx.y + FILTER_SIZE / 2;

    float sigma = 1;
    float kernel_weight = 0;

// Записываем в sharedMem цвета точек вокруг (x, y) и (x, y) (когда dx=0 dy=0)
    for (int dy = -FILTER_SIZE / 2; dy <= FILTER_SIZE / 2; dy++) {
      for (int dx = -FILTER_SIZE / 2; dx <= FILTER_SIZE / 2; dx++) {
        int sharedMemX = sharedX + dx;
        int sharedMemY = sharedY + dy;
        int globalX = min(max(x + dx, 0), width - 1);
        int globalY = min(max(y + dy, 0), height - 1);
        for(int c = 0; c < channels; c++) {
          sharedMem[(sharedMemY * sharedWidth + sharedMemX) * channels + c] = input[(globalY * width + globalX) * channels + c];
        }
        kernel_weight += GAUSS(dx, sigma) * GAUSS(dy, sigma);
      }
    }

    __syncthreads();

    if (x < width && y < height) {
        for (int c = 0; c < channels; ++c) {
            int color = 0;
            for (int dy = -FILTER_SIZE / 2; dy <= FILTER_SIZE / 2; dy++) {
                for (int dx = -FILTER_SIZE / 2; dx <= FILTER_SIZE / 2; dx++) {
                    int shiftedX = sharedX + dx;
                    int shiftedY = sharedY + dy;

                    float factor = GAUSS(dx, sigma) * GAUSS(dy, sigma) / kernel_weight;

                    if (shiftedX >= 0 && shiftedX < sharedWidth && shiftedY >= 0 && shiftedY < sharedHeight) {
                        color += factor * sharedMem[(shiftedY * sharedWidth + shiftedX) * channels + c];
                    }
                }
            }

            // Запись результата обратно в глобальную память
            output[(y * width + x) * channels + c] = color;
        }

    }
}

// Размытие с использованием textured memory
__global__ void blurFilterTexture(cudaTextureObject_t texObj, unsigned char* output, int width, int height, int channels) {

    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {

      float sigma = 1;
      float kernel_weight = 0;

    for (int dy = -FILTER_SIZE / 2; dy <= FILTER_SIZE / 2; dy++) {
          for (int dx = -FILTER_SIZE / 2; dx <= FILTER_SIZE / 2; dx++) {
              kernel_weight += GAUSS(dx, sigma) * GAUSS(dy, sigma);
          }
      }

      int color[3] = {0, 0, 0};

      for (int dy = -FILTER_SIZE / 2; dy <= FILTER_SIZE / 2; dy++) {
          for (int dx = -FILTER_SIZE / 2; dx <= FILTER_SIZE / 2; dx++) {
               int sampledX = x + dx;
               int sampledY = y + dy;

               if (sampledX >= 0 && sampledX < width && sampledY >= 0 && sampledY < height) {

                  float factor = GAUSS(dx, sigma) * GAUSS(dy, sigma) / kernel_weight;

                  uchar4 sampledValue = tex2D<uchar4>(texObj, sampledX, sampledY);
                  color[0] += factor * sampledValue.x;
                  color[1] += factor * sampledValue.y;
                  color[2] += factor * sampledValue.z;
               }
          }
       }

        for (int c = 0; c < channels; c++) {
            output[(y * width + x) * channels + c] = min(255, color[c]);
        }
        output[(y * width + x) * channels + 3] = 255; // Альфа-канал
    }

}

// Загрузка изображения
void loadImage(const std::string& filename, cv::Mat& image) {
    image = cv::imread(filename, cv::IMREAD_UNCHANGED);
    if (image.empty()) {
        std::cerr << "Image is empty " << filename << std::endl;
        exit(EXIT_FAILURE);
    }
    std::cout << "Image is loaded. Size: " << image.cols << "x" << image.rows << ", Channels num: " << image.channels() << std::endl;
}

// Сохранение изображения
void saveImage(const std::string& filename, const cv::Mat& image) {
    if (!cv::imwrite(filename, image)) {
        std::cerr << "The error during saving image " << filename << std::endl;
        exit(EXIT_FAILURE);
    }
    std::cout << "Output image was saved successfully " << filename << std::endl;
}

// Выделение памяти и копирование данных на GPU
void allocateAndCopyToDevice(unsigned char* h_data, unsigned char** d_data, size_t size) {
    CUDA_CHECK_ERROR(cudaMalloc(d_data, size));
    CUDA_CHECK_ERROR(cudaMemcpy(*d_data, h_data, size, cudaMemcpyHostToDevice));
}

int main() {
    const std::string filename = "image.jpg";
    cv::Mat image;
    loadImage(filename, image);

    // Преобразование изображения в формат RGBA
    if (image.channels() == 3) {
        cv::cvtColor(image, image, cv::COLOR_BGR2BGRA);
    }

    int width = image.cols;
    int height = image.rows;
    int channels = image.channels();

    size_t imageSize = width * height * channels * sizeof(unsigned char);

   // -- Shared --

    // Объявление указателей для данных на GPU
    unsigned char* d_input;
    unsigned char* d_output;

    // Выделение и копирование данных на GPU
    allocateAndCopyToDevice(image.data, &d_input, imageSize);
    CUDA_CHECK_ERROR(cudaMalloc(&d_output, imageSize));

    // Настройка размера блока и сетки
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize((width + TILE_SIZE - 1) / TILE_SIZE, (height + TILE_SIZE - 1) / TILE_SIZE);

    size_t sharedMemSize = (BLOCK_SIZE + FILTER_SIZE - 1) * (BLOCK_SIZE + FILTER_SIZE - 1) * channels * sizeof(unsigned char);

    auto sharedStart = std::chrono::high_resolution_clock::now();
    blurFilterShared<<<gridSize, blockSize, sharedMemSize>>>(d_input, d_output, width, height, channels);
    CUDA_CHECK_ERROR(cudaDeviceSynchronize());
    auto sharedEnd = std::chrono::high_resolution_clock::now();
    std::cout << "Time spend for Shared Memory (Blur filter) " << ": " << std::chrono::duration_cast<std::chrono::milliseconds>(sharedEnd - sharedStart).count() << " ms" << std::endl;

    // Копирование результата обратно на хост
    CUDA_CHECK_ERROR(cudaMemcpy(image.data, d_output, imageSize, cudaMemcpyDeviceToHost));

    saveImage("./output_shared_blur.png", image);

    cudaFree(d_input);
    cudaFree(d_output);

   // -- Texture --

     // Создание текстуры для входных данных
    cudaChannelFormatDesc channelDesc = cudaCreateChannelDesc<uchar4>();
    cudaArray* cuArray;
    CUDA_CHECK_ERROR(cudaMallocArray(&cuArray, &channelDesc, width, height));

    // Копирование данных в массив с использованием cudaMemcpy2DToArray
    CUDA_CHECK_ERROR(cudaMemcpy2DToArray(cuArray, 0, 0, image.data, width * channels * sizeof(unsigned char), width * channels * sizeof(unsigned char), height, cudaMemcpyHostToDevice));

    // Создание текстурного объекта
    cudaResourceDesc resDesc = {};
    resDesc.resType = cudaResourceTypeArray;
    resDesc.res.array.array = cuArray;

    cudaTextureDesc texDesc = {};
    texDesc.addressMode[0] = cudaAddressModeClamp;
    texDesc.addressMode[1] = cudaAddressModeClamp;
    texDesc.filterMode = cudaFilterModePoint;
    texDesc.readMode = cudaReadModeElementType;
    texDesc.normalizedCoords = false;

    cudaTextureObject_t texObj = 0;
    CUDA_CHECK_ERROR(cudaCreateTextureObject(&texObj, &resDesc, &texDesc, nullptr));

    // Выделение памяти для выходных данных
    CUDA_CHECK_ERROR(cudaMalloc(&d_output, imageSize));

    // Запуск CUDA-ядра
    auto textureStart = std::chrono::high_resolution_clock::now();
    blurFilterTexture<<<gridSize, blockSize>>>(texObj, d_output, width, height, channels);
    CUDA_CHECK_ERROR(cudaDeviceSynchronize());
    auto textureEnd = std::chrono::high_resolution_clock::now();
    std::cout << "Time spend for Textured Memory (Blur filter)" << ": " << std::chrono::duration_cast<std::chrono::milliseconds>(textureEnd - textureStart).count() << " ms" << std::endl;

    // Копирование результата обратно на хост
    cv::Mat outputImage(height, width, CV_8UC4);
    CUDA_CHECK_ERROR(cudaMemcpy(outputImage.data, d_output, imageSize, cudaMemcpyDeviceToHost));

    saveImage("./output_textured_blur.png", outputImage);

    // Освобождение ресурсов
    CUDA_CHECK_ERROR(cudaDestroyTextureObject(texObj));
    CUDA_CHECK_ERROR(cudaFreeArray(cuArray));
    CUDA_CHECK_ERROR(cudaFree(d_output));

    return 0;
}

Overwriting example.cu


In [ ]:
!nvcc -o example example.cu -o cv -I/usr/include/opencv4 -lopencv_core -lopencv_imgproc -lopencv_highgui -lopencv_imgcodecs

nvcc warning : incompatible redefinition for option 'output-file', the last value of this option was used
/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitchi

In [ ]:
!./cv

Image is loaded. Size: 900x590, Channels num: 3
Time spend for Shared Memory (Blur filter) : 2 ms
Output image was saved successfully ./output_shared_blur.png
Time spend for Textured Memory (Blur filter): 1 ms
Output image was saved successfully ./output_textured_blur.png
